In [1]:
import pandas as pd
import os

In [2]:
def load_multiple_csv(folder_path):
    dfs = []

    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            df = pd.read_csv(file_path, parse_dates=['Date time'])
            dfs.append(df)
    combined_df = pd.concat(dfs)
    return combined_df

In [3]:
def PgeRate(row):
    if row['Date time'].hour >= 16 and row['Date time'].hour < 21:
        return 0.46357
    elif row['Date time'].hour in [15, 21, 22, 23]:
        return 0.44687
    else:
        return 0.27818

In [4]:
dfs = load_multiple_csv('/Users/darrenkoh/Tesla Energy/Nov')
sorted_dfs = dfs.sort_values(by='Date time')
print(sorted_dfs.dtypes)
sorted_dfs.head()

Date time          object
Home (kW)         float64
Vehicle (kW)      float64
Powerwall (kW)    float64
Solar (kW)        float64
Grid (kW)         float64
dtype: object


,Date time,Home (kW),Vehicle (kW),Powerwall (kW),Solar (kW),Grid (kW)
0,2023-11-01 00:00:00-07:00,6.3,0.0,6.3,0.0,0.0
1,2023-11-01 00:05:00-07:00,6.2,0.0,6.2,0.0,0.0
2,2023-11-01 00:10:00-07:00,4.1,0.0,4.1,0.0,0.0
3,2023-11-01 00:15:00-07:00,2.6,0.0,2.6,0.0,0.0
4,2023-11-01 00:20:00-07:00,4.2,0.0,4.2,0.0,0.0


In [5]:
# Add PGE TOU Rate colums
sorted_dfs['Peak'] = sorted_dfs.apply(PgeRate, axis=1)
sorted_dfs.head()

,Date time,Home (kW),Vehicle (kW),Powerwall (kW),Solar (kW),Grid (kW),Peak
0,2023-11-01 00:00:00-07:00,6.3,0.0,6.3,0.0,0.0,0.27818
1,2023-11-01 00:05:00-07:00,6.2,0.0,6.2,0.0,0.0,0.27818
2,2023-11-01 00:10:00-07:00,4.1,0.0,4.1,0.0,0.0,0.27818
3,2023-11-01 00:15:00-07:00,2.6,0.0,2.6,0.0,0.0,0.27818
4,2023-11-01 00:20:00-07:00,4.2,0.0,4.2,0.0,0.0,0.27818


In [8]:
sorted_dfs['cost'] = (sorted_dfs['Home (kW)'] + sorted_dfs['Vehicle (kW)']) * sorted_dfs['Peak']
print(sorted_dfs.head())
print(sum(sorted_dfs['cost']))

                   Date time  Home (kW)  Vehicle (kW)  Powerwall (kW)  \
0  2023-11-01 00:00:00-07:00        6.3           0.0             6.3   
1  2023-11-01 00:05:00-07:00        6.2           0.0             6.2   
2  2023-11-01 00:10:00-07:00        4.1           0.0             4.1   
3  2023-11-01 00:15:00-07:00        2.6           0.0             2.6   
4  2023-11-01 00:20:00-07:00        4.2           0.0             4.2   

   Solar (kW)  Grid (kW)     Peak      cost  
0         0.0        0.0  0.27818  1.752534  
1         0.0        0.0  0.27818  1.724716  
2         0.0        0.0  0.27818  1.140538  
3         0.0        0.0  0.27818  0.723268  
4         0.0        0.0  0.27818  1.168356  
11804.183986000016


In [9]:
sorted_dfs.to_csv('/Users/darrenkoh/Tesla Energy/Nov.csv', index=False)